# RealNVP flows from scratch

__Objective:__ build and train a simple RealNVP flow model from scratch.

__Source:__ D. Foster, [_Generative deep learning_](https://www.oreilly.com/library/view/generative-deep-learning/9781492041931/) (2nd ed.) (with notebooks [here](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition)).

**Setup:**
- We start from a vetor $z \in \mathbb{R}^D$ in latent space, which we sample from a multivariate standard normal distribution, so $p_Z \sim \mathcal{N}(0, I)$.
- We transform $z$ to the "real" data space $x \in \mathbb{R}^D$ via the RealNVP transformation so that $z \to x = x(z)$ is the **forward** transformation (this is opposite to what's done in the source, in which this is taken to be the inverse transformation, but for RealNVP's it doesn't really matter the forward and the inverse transformation are computationally equivalent).
- The RealNVP transformation is implemented by a stack of **coupling layers** with feature permutation operations (bijectors) in between.
- Following the RealNVP recipe, in each coupling layer the first $d$ dimensions (features) of $x$ are singled out and used to generate the corresponding dimensions of $z$ (an identity transformation) and to parametrize (via a neural network) an affine transformation for the last $(D - d)$ dimensions of $z$.
- Full transformation for a single coupling layer:
$$
\begin{array}{lll}
z_i &=& x_i\quad \forall x=i, \ldots, d\\
z_j &=& x_j\,\exp\left( s_j(x_1, \ldots, x_d) \right) + t_j(x_1, \ldots, x_d)\quad \forall j = d+1, \ldots, D
\end{array}
$$
where the vectors $s, t \in \mathbb{R}^{D-d}$ are the tensors outputted by the coupling layer and are functions of $x_1, \ldots, x_d$ given by a neural network.

In [ ]:
import sys
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns

sys.path.append('../modules/')

tfd = tfp.distributions

sns.set_theme()

%load_ext autoreload
%autoreload 2

## Coupling layer

The coupling layer is responsible for taking the first $d$ dimensions (features) of the input and outputting a scale and a translation tensor (so two outputs) to be used to parametrize an affine transformation for the remaining $(D - d)$ dimensions of the input.

In [ ]:
from real_nvp import CouplingLayer

In [ ]:
test_cl = CouplingLayer(
    n_masked_dims=2,
    n_affine_dims=3,
    hidden_layers_dims=[32, 32]
)

s, t = test_cl(tf.random.normal(shape=(14, 5)))

## RealNVP bijector [WIP]

Parametrize an affine (scale and then shift) tranformation with the output from the `CouplingLayer`.

In [ ]:
from real_nvp import RealNVPBijector

In [ ]:
test_data = tf.ones(shape=(4, 5)) * 2.4

In [ ]:
test_real_nvp_bij = RealNVPBijector(test_cl)

In [ ]:
test_real_nvp_bij.forward(test_data), test_real_nvp_bij.inverse(test_data)

Check a "cycle condition": applying the forward and then the inverse transformation on some data (and vice versa) we should reobtain the starting tensors.

In [ ]:
test_affine_bij.inverse(test_real_nvp_bij.forward(test_data))

In [ ]:
test_affine_bij.forward(test_real_nvp_bij.inverse(test_data))

## RealNVP model [WIP]

Implement the RealNVP flow model as a Keras `Model` object.